In [1]:
#import subprocess
#proc = subprocess.run(["python", "./main.py"],stdout = subprocess.PIPE, stderr = subprocess.PIPE)
#print(proc.stdout.decode("utf8"))
#print(proc.stderr.decode("utf8"))

In [2]:
import instances
Customers = instances.Customers
Points = instances.Points
C, F = instances.C, instances.F
tour = instances.tour

instance_name = "C" + str(len(Customers)-1)

In [3]:
import inputs
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq = inputs.make_inputs(tour, Customers, Points, F, C)

Preceding constraints are done.
Time-window constarints are done.
Pick-up and delivery constarints are done.
Charge constarints are done.


In [4]:
# Gurobiによって最適解を求める
import formulation
import importlib
importlib.reload(formulation)
P = formulation.solve_primal(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, instance_name+"P", len(tour))

Academic license - for non-commercial use only - expires 2021-08-18
Using license file /Users/hasimoto/gurobi.lic
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 15792 rows, 15005 columns and 38370 nonzeros
Model fingerprint: 0x6b14ba4a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Presolve removed 5191 rows and 4397 columns
Presolve time: 0.04s
Presolved: 10601 rows, 10608 columns, 33771 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1602325e+01   1.163955e-01   2.999992e+06      0s
      12    1.5766775e+07   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.08 seconds
Optimal objective  1.576677518e+07


In [5]:
CBasis = [constr.CBasis for constr in P.getConstrs()]
VBasis = [var.VBasis for var in P.getVars()]

In [6]:
def solve_primal2(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, instance_name, num_vars,VBasis,CBasis):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    s_i : 顧客iへ車両が到着する時刻を表す変数
    r_i : 顧客 i における車両の燃料補充時間を表す変数
    c_i : 顧客 i に到着した時の燃料残量を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    q_i : 顧客 i の荷物を配達するまでの制限時間に関する違反量を表す変数
    """
    s = m.addMVar(shape=num_vars, lb=0.0,  vtype=GRB.CONTINUOUS, name="s")
    r = m.addMVar(shape=num_vars, lb=0.0,  vtype=GRB.CONTINUOUS, name="r")
    c = m.addMVar(shape=num_vars, lb=0.0,  vtype=GRB.CONTINUOUS, name="c")
    p = m.addMVar(shape=num_vars, lb=0.0,  vtype=GRB.CONTINUOUS, name="p")
    q = m.addMVar(shape=num_vars, lb=0.0,  vtype=GRB.CONTINUOUS, name="q")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(op.T @ p + oq.T @ q, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    ## 各係数行列のサイズを合わせる
    m.addConstr(As @ s + Ac_leq @ c + Ar_leq @ r + Ap @ p + Aq @ q <= b_leq, name="c_leq")
    m.addConstr(Ac_eq @ c + Ar_eq @ r == b_eq, name="c_eq")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # パラメータ
    m.Params.Presolve = 0
    m.Params.Method = 1

    
    for i, var in enumerate(m.getVars()):
#        var.PStart = PStarts[i]
        var.VBasis = VBasis[i]
    for i, constr in enumerate(m.getConstrs()):
        constr.CBasis = CBasis[i]

    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
    
    # モデルをテキストファイルにする
#    m.write("out"+instance_name+".json")
        
    return m

In [8]:
solve_primal2(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, op, oq, instance_name+"P", len(tour),VBasis,CBasis)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 15792 rows, 15005 columns and 38370 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5766775e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.576677518e+07


<gurobi.Model Continuous instance LP_for_VRPC3000P: 15792 constrs, 15005 vars, Parameter changes: Method=1, Presolve=0>